# Quantum codes do not fix qubit independent errors
#### Authors: J. Lacalle, L. M. Pozo-Coronado, A. L. Fonseca de Oliveira, R. Martin-Cuevas

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

## 1. Finding the basis associated with the 5-qubit quantum code